1. Importa las librerías requeridas

In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.decomposition import PCA

2.	Lee el archivo CSV llamado empleadosRETO.csv y coloca los datos en un frame de Pandas llamado EmpleadosAttrition.

In [2]:
EmpleadosAttrition = pd.read_csv(
    r'C:\Users\alanf\OneDrive\Documentos\Data Scientist\Ingeniería de las características\empleadosRETO.csv')


3.	Elimina las columnas que, con alta probabilidad (estimada por ti), no tienen relación alguna con la salida. Hay algunas columnas que contienen información que no ayuda a definir el desgaste de un empleado, tal es caso de las siguientes:

a.	EmployeeCount: número de empleados, todos tienen un 1

b.	EmployeeNumber: ID del empleado, el cual es único para cada empleado

c.	Over18: mayores de edad, todos dicen “Y”

d.	StandardHours: horas de trabajo, todos tienen “80”


In [3]:
EmpleadosAttrition.drop(['EmployeeCount', 'EmployeeNumber',
                         'Over18', 'StandardHours'], axis=1, inplace=True)


4.	Analizando la información proporcionada, detectaste que no se cuenta con los años que el empelado lleva en la compañía y parece ser un buen dato. Dicha cantidad se puede calcular con la fecha de contratación ‘HiringDate’:

a.	Crea una columna llamada Year y obtén el año de contratación del empleado a partir de su fecha ‘HiringDate’. No se te olvide que debe ser un entero.

b.	Crea una columna llamada YearsAtCompany que contenga los años que el empleado lleva en la compañía hasta el año 2018. Para su cálculo, usa la variable Year que acabas de crear.


In [4]:
#Crear columna Year
#Al tratar te crear la columna Year, nos muestra un error en la posición 229,
#el cual tiene una fecha de 2/30/2012
#indicando 30 de febrero, lo cual no tiene sentido así
# que se decidió por eliminar el renglón
EmpleadosAttrition.drop(229, axis=0, inplace=True)
EmpleadosAttrition['Year'] = pd.to_datetime(EmpleadosAttrition['HiringDate'],
                                            dayfirst=False).dt.year

#Crear columna YearsAtCompany
EmpleadosAttrition['YearsAtCompany'] = 2018 - EmpleadosAttrition['Year']

display(EmpleadosAttrition[['HiringDate', 'Year', 'YearsAtCompany']].head())


,HiringDate,Year,YearsAtCompany
0,06/06/2013,2013,5
1,12/25/2015,2015,3
2,2/14/2017,2017,1
3,7/29/2010,2010,8
4,07/10/2011,2011,7


5.	La DistanceFromHome está dada en kilómetros, pero tiene las letras “km” al final y así no puede ser entera:

a.	Renombra la variable DistanceFromHome a DistanceFromHome_km.

b.	Crea una nueva variable DistanceFromHome que sea entera, es decir, solo con números.


In [5]:
#Renombrar variable DistanceFromHome
EmpleadosAttrition.rename(columns={'DistanceFromHome':'DistanceFromHome_km'}, inplace=True)

#Crear columna DistanceFromHome
EmpleadosAttrition['DistanceFromHome'] = EmpleadosAttrition['DistanceFromHome_km'].str.replace(' km', '')
EmpleadosAttrition[['DistanceFromHome']].head()

,DistanceFromHome
0,1
1,6
2,7
3,7
4,15


6.	Borra las columnas Year, HiringDate y DistanceFromHome_km debido a que ya no son útiles.

In [6]:
EmpleadosAttrition.drop(['Year', 'HiringDate', 'DistanceFromHome_km'], 
                        axis=1, inplace=True)
EmpleadosAttrition.head()

,Age,BusinessTravel,Department,Education,EducationField,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,...,PerformanceRating,RelationshipSatisfaction,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsInCurrentRole,YearsSinceLastPromotion,Attrition,YearsAtCompany,DistanceFromHome
0,50,Travel_Rarely,Research & Development,2,Medical,4,Male,3,4,Research Director,...,4,3,32,1,2,4,1,No,5,1
1,36,Travel_Rarely,Research & Development,2,Medical,2,Male,3,2,Manufacturing Director,...,4,4,7,0,3,2,0,No,3,6
2,21,Travel_Rarely,Sales,1,Marketing,2,Male,3,1,Sales Representative,...,3,2,1,3,3,0,1,Yes,1,7
3,52,Travel_Rarely,Research & Development,4,Life Sciences,2,Male,3,3,Healthcare Representative,...,3,4,18,4,3,6,4,No,8,7
4,33,Travel_Rarely,Research & Development,1,Medical,2,Male,3,3,Manager,...,3,4,15,2,4,6,7,Yes,7,15


7.	Aprovechando los ajustes que se están haciendo, la empresa desea saber si todos los departamentos tienen un ingreso promedio similar. Genera una nuevo frame llamado SueldoPromedioDepto que contenga el MonthlyIncome promedio por departamento de los empleados y colócalo en una variable llamada SueldoPromedio. Esta tabla solo es informativa, no la vas a utilizar en el set de datos que estás construyendo.

In [7]:
SueldoPromedioDepto = EmpleadosAttrition.groupby('Department')['MonthlyIncome'].mean()
SueldoPromedioDepto = SueldoPromedioDepto.reset_index()
SueldoPromedioDepto.rename(columns={'MonthlyIncome':'SueldoPromedio'}, 
                           inplace=True)
SueldoPromedioDepto

,Department,SueldoPromedio
0,Human Resources,6239.888889
1,Research & Development,6804.149813
2,Sales,7192.609756


8.	La variable MonthlyIncome tiene un valor numérico muy grande comparada con las otras variables. Escala dicha variable para que tenga un valor entre 0 y 1.

In [8]:
EmpleadosAttrition['MonthlyIncome']=(EmpleadosAttrition['MonthlyIncome']/
                                    EmpleadosAttrition['MonthlyIncome'].abs().max())
EmpleadosAttrition[['MonthlyIncome']].head()

,MonthlyIncome
0,0.871126
1,0.247384
2,0.134131
3,0.522956
4,0.681420


9.	Todo parece indicar que las variables categóricas que quedan sí son importantes para obtener la variable de salida. Convierte todas las variables categóricas que quedan a numéricas:

a.	BusinessTravel

b.	Department

c.	EducationField

d.	Gender

e.	JobRole

f.	MaritalStatus

g.	Attrition


In [9]:
# La instrucción no lo menciona, pero también incluí la variable OverTime para ser convertida a numérica.
EmpleadosAttrition = pd.get_dummies(EmpleadosAttrition, columns=['BusinessTravel',
                                                                 'OverTime',
                                                                 'Department', 
                                                                 'EducationField', 
                                                                 'Gender', 
                                                                 'JobRole', 
                                                                 'MaritalStatus', 
                                                                 'Attrition'])
#EmpleadosAttrition.columns
#EmpleadosAttrition.head()
with pd.option_context('display.max_columns', None):
    display(EmpleadosAttrition.head())

,Age,Education,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsInCurrentRole,YearsSinceLastPromotion,YearsAtCompany,DistanceFromHome,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,OverTime_No,OverTime_Yes,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Attrition_No,Attrition_Yes
0,50,2,4,3,4,4,0.871126,9,22,4,3,32,1,2,4,1,5,1,False,False,True,True,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,True,False
1,36,2,2,3,2,2,0.247384,6,20,4,4,7,0,3,2,0,3,6,False,False,True,True,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,True,False
2,21,1,2,3,1,2,0.134131,1,13,3,2,1,3,3,0,1,1,7,False,False,True,True,False,False,False,True,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,True
3,52,4,2,3,3,2,0.522956,7,19,3,4,18,4,3,6,4,8,7,False,False,True,True,False,False,True,False,False,True,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,True,False
4,33,1,2,3,3,3,0.681420,7,12,3,4,15,2,4,6,7,7,15,False,False,True,False,True,False,True,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,True


10.	Ahora debes hacer la evaluación de las variables para quedarte con las mejores. Calcula la correlación lineal de cada una de las variables con respecto al Attrition.

In [10]:
correlation_with_attrition=EmpleadosAttrition.corr(numeric_only=True)['Attrition_Yes']
display(correlation_with_attrition)

Age                                 -0.212459
Education                           -0.054478
EnvironmentSatisfaction             -0.125091
JobInvolvement                      -0.168047
JobLevel                            -0.214486
JobSatisfaction                     -0.163964
MonthlyIncome                       -0.195044
NumCompaniesWorked                  -0.008590
PercentSalaryHike                   -0.061714
PerformanceRating                   -0.007010
RelationshipSatisfaction            -0.032777
TotalWorkingYears                   -0.214043
TrainingTimesLastYear               -0.071592
WorkLifeBalance                     -0.023063
YearsInCurrentRole                  -0.204453
YearsSinceLastPromotion             -0.069835
YearsAtCompany                      -0.176287
BusinessTravel_Non-Travel           -0.101103
BusinessTravel_Travel_Frequently     0.034833
BusinessTravel_Travel_Rarely         0.043551
OverTime_No                         -0.324332
OverTime_Yes                      

11.	Selecciona solo aquellas variables que tengan una correlación mayor o igual a 0.1, dejándolas en otro frame llamado EmpleadosAttritionFinal. No olvides mantener la variable de salida Attrition; esto es equivalente a borrar las que no cumplen con el límite.

In [11]:
EmpleadosAttritionFinal=EmpleadosAttrition[correlation_with_attrition[correlation_with_attrition >= 0.1].index]

EmpleadosAttritionFinal

,OverTime_Yes,EducationField_Technical Degree,JobRole_Laboratory Technician,JobRole_Sales Representative,MaritalStatus_Single,Attrition_Yes
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,True,True,True
3,False,False,False,False,True,False
4,True,False,False,False,False,True
...,...,...,...,...,...,...
395,True,False,True,False,False,True
396,True,False,False,False,False,True
397,True,False,False,False,False,False
398,False,False,True,False,False,False


12.	Crea una nueva variable llamada EmpleadosAttritionPCA formada por los componentes principales del frame EmpleadosAttritionFinal. Recuerda que el resultado del proceso PCA es un numpy array, por lo que, para hacer referencia a una columna, por ejemplo, la 0, puedes usar la instrucción EmpleadosAttritionPCA[:,0]).

In [12]:
# Crear variable EmpleadodAttritionPCA
EmpleadosAttritionPCA = PCA()

#Agregar datos al PCA
EmpleadosAttritionPCA.fit(EmpleadosAttritionFinal)


# Imprimir componentes y varianzas
print("Components:")
print(EmpleadosAttritionPCA.components_)
print("Explained variance:")
print(EmpleadosAttritionPCA.explained_variance_)
print("Explained variance ratio:")
print(EmpleadosAttritionPCA.explained_variance_ratio_)

# Transformar datos
EmpleadosAttritionPCA = EmpleadosAttritionPCA.fit_transform(EmpleadosAttritionFinal)
EmpleadosAttritionPCA

Components:
[[ 0.65140538  0.03491614 -0.00292128  0.08604259  0.54793091  0.51653824]
 [-0.59107211 -0.04946417 -0.12582237  0.11471714  0.78061975 -0.09913916]
 [-0.2379893  -0.02299944  0.92203756 -0.0390382   0.01101286  0.30171774]
 [-0.38480864  0.2273103  -0.31011134  0.33378069 -0.28518703  0.71508188]
 [ 0.13111822  0.61098262  0.19024039  0.67342267  0.02585696 -0.34518136]
 [-0.06630395  0.75553653 -0.04057701 -0.64265467  0.09101104  0.04282297]]
Explained variance:
[0.24575931 0.19521691 0.14748056 0.09992768 0.05287977 0.04637068]
Explained variance ratio:
[0.31202185 0.24785202 0.18724483 0.12687056 0.06713742 0.05887332]


array([[-0.41938778, -0.02559025, -0.14026445,  0.08287468, -0.09308544,
        -0.00919999],
       [-0.41938778, -0.02559025, -0.14026445,  0.08287468, -0.09308544,
        -0.00919999],
       [ 0.73112396,  0.77060748,  0.13342795,  0.84655022,  0.26101282,
        -0.51802065],
       ...,
       [ 0.2320176 , -0.61666236, -0.37825375, -0.30193396,  0.03803277,
        -0.07550394],
       [-0.42230906, -0.15141262,  0.7817731 , -0.22723666,  0.09715494,
        -0.049777  ],
       [-0.41938778, -0.02559025, -0.14026445,  0.08287468, -0.09308544,
        -0.00919999]])

13.	Agrega el mínimo número de Componentes Principales en columnas del frame EmpleadosAttritionPCA que logren explicar el 80% de la varianza, al frame EmpleadosAttritionFinal. Puedes usar la instrucción assign, columna por columna, llamando a cada una C0, C1, etc., hasta las que vayas a agregar.

In [13]:
# Inicializar PCA y agregar datos
pca = PCA()
pca.fit(EmpleadosAttritionFinal)

# Calcular el radio de varianza explicado y calcular la suma acumulada
cumulative_variance_ratio = pca.explained_variance_ratio_.cumsum()

# Encontrar los componentes que expliquen al menos el 80% de la varianza
n_components_80_percent = np.where(cumulative_variance_ratio >= 0.80)[0][0] + 1
print(f"Mínimo número de componentes para explicar 80% varianza:{n_components_80_percent}")

# Transformar los datos
EmpleadosAttritionPCA = pca.transform(EmpleadosAttritionFinal)

# Añadir los componentes principales seleccionados al DataFrame EmpleadosAttritionFinal
for i in range(n_components_80_percent):
    EmpleadosAttritionFinal = EmpleadosAttritionFinal.assign(**{f'C{i}': EmpleadosAttritionPCA[:, i]})

EmpleadosAttritionFinal

Mínimo número de componentes para explicar 80% varianza:4


,OverTime_Yes,EducationField_Technical Degree,JobRole_Laboratory Technician,JobRole_Sales Representative,MaritalStatus_Single,Attrition_Yes,C0,C1,C2,C3
0,False,False,False,False,False,False,-0.419388,-0.025590,-0.140264,0.082875
1,False,False,False,False,False,False,-0.419388,-0.025590,-0.140264,0.082875
2,False,False,False,True,True,True,0.731124,0.770607,0.133428,0.846550
3,False,False,False,False,True,False,0.128543,0.755029,-0.129252,-0.202312
4,True,False,False,False,False,True,0.748556,-0.715802,-0.076536,0.413148
...,...,...,...,...,...,...,...,...,...,...
395,True,False,True,False,False,True,0.745635,-0.841624,0.845502,0.103037
396,True,False,False,False,False,True,0.748556,-0.715802,-0.076536,0.413148
397,True,False,False,False,False,False,0.232018,-0.616662,-0.378254,-0.301934
398,False,False,True,False,False,False,-0.422309,-0.151413,0.781773,-0.227237


14.	Guarda el set de datos que has formado y que tienes en EmpleadosAttritionFinal en un archivo CSV llamado EmpleadosAttritionFinal.csv. Las últimas columnas que colocaste quedarán después de la variable Attrition, lo cual no importa, pero si gustas lo puedes arreglar antes de escribir el archivo.

In [14]:
#El archivo se creará automáticamente en la carpeta en donde 
# está el archivo .ipynb
EmpleadosAttritionFinal.to_csv('EmpleadosAttritionFinal.csv', index=False)